# Web Search Engine
## for the mathematics department at Brown University

by Semin Kim. Last Update: 2017/01/05.

We implement a web search engine for the department of mathematics at Brown University.

1. Crawler
    * Use BeautifulSoup for HTML parsing.
    * BFS through link information. 
    * Visit pages whose hostname ends with 'math.brown.edu'.
    * Scrape only text pages and skip pages like pdf and image files.  
1. Indexing
    * Use SQLite for a database. 
    * Use an inverted index for a schema. 
1. Word Process
    * Use NLTK.
    * Apply the Porter stemming and remove English stopwords for words in indexing and querying.
    * Use only alphanumeric character for simplicity. (limitation: 'info@math.brown.edu' is parsed as 'info math brown edu')
1. Search and Ranking
    * Retrieve pages that contain every word in a query (after stemming and removing stopwords).
    * Use the PageRank algorithm to rank matched pages.
    * To maintain the Markov property, transfer the probability of links toward an outside of math.brown.edu to the original page. 
1. TODO
    * Improve the ranking algorithm by combining methods, such as word frequency, word location, and link text together with the  PageRank algorithm with appropriate weights. 
    * Use id instead of raw data for database space efficiency.
    * Add a feedback routine such as learning from click to improve ranking. 

*Reference*
* Zhai, C & Massung, S. (2016). *Text Data Management and Analysis*. ACM Books.
* Raschka, S. (2015). *Python Machine Learning*. Packt Publishing. 
* Segaran, T. (2007). *Programming Collective Intelligence*. O'Reilly.

In [1]:
import sqlite3, re, time, sys 
from urllib.request import urlopen
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup, Comment, Doctype
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import pandas as pd
import numpy as np

In [2]:
# All SQL queries implemented in DBController only. 
# DBController provides easy api to access DB, and also encapsulate DB schema. 
class DBController:
    def __init__(self):
        db_name = 'search_engine.db'
        self.__db_name = db_name
        self.__con = sqlite3.connect(db_name)
        
    def __del__(self):
        self.__con.commit()
        self.__con.close()
    
    def initDB(self):
        table_names = ['doc_list', 'word_list', 'word_posting', 'link_list', 
                        'pagerank_list', 'error_doc_list']
        for table in table_names:
            cur = self.__con.cursor()
            if cur.execute("select * from sqlite_master \
                           where name ='%s' and type='table'" %table).fetchone():
                self.__con.execute('drop table %s' %table)

        self.__con.execute("create table doc_list(doc)")
        self.__con.execute("create table word_list(word, n_doc, total_freq)")
        self.__con.execute("create table word_posting(word_id, doc_id, freq, position)")
        self.__con.execute("create table link_list(from_doc_id, to_doc_id, link_text)")
        self.__con.execute("create table pagerank_list(doc_id, pagerank)")
        self.__con.execute("create table error_doc_list(doc, error_msg)")
    
    def isDocIndexed(self, url):
        cur = self.__con.cursor()
        row = cur.execute("select rowid, doc from doc_list \
                           where doc = '%s'" % url).fetchone()
        if row:
            return True
        else:
            return False
        
    def addDoc(self, url):
        if self.isDocIndexed(url):
            return
        self.__con.execute("insert into doc_list (doc) values ('%s')" %url)
        
    def addWord(self, word, url, freq, position):
        cur = self.__con.cursor()
        
        rowid = None
        n_doc = 0
        total_freq = 0
        
        # add word to word_list
        # If word exists, update counts.  
        row = cur.execute("select rowid, word, n_doc, total_freq from word_list \
                            where word = '%s'"% word).fetchone()
        if row:
            rowid = row[0]
            n_doc = row[2]
            total_freq = row[3]
        else:
            rowid = self.__con.execute("insert into word_list (word, n_doc, total_freq) \
                                        values ('%s',0,0)" 
                                        %(word)).lastrowid
        
        self.__con.execute("update word_list set n_doc=%d, total_freq=%d \
                            where word='%s'"
                            %(n_doc+1, total_freq+freq, word))
        
        # add word to word_posting. 
        self.__con.execute("insert into word_posting (word_id, doc_id, freq, position) \
                            values ('%s', '%s', %d, '%s')"
                            %(word, url, freq, position))
        
    def addLink(self, from_url, to_url, link_text):
        self.__con.execute("insert into link_list(from_doc_id, to_doc_id, link_text) \
                            values ('%s', '%s', '%s')" 
                            %(from_url, to_url, link_text))
  
    def isErrorDoc(self, url):
        cur = self.__con.cursor()
        row = cur.execute("select rowid, doc from error_doc_list \
                           where doc = '%s'" % url).fetchone()
        if row:
            return True
        else:
            return False
        
    def addErrorDoc(self, url, error_msg=''):
        if self.isErrorDoc(url):
            return
        # add url to error_doc_list
        self.__con.execute("insert into error_doc_list (doc, error_msg) \
                            values ('%s', '%s')" %(url, error_msg))
    
    def getDocs(self, word):
        # return list of documents that contain a single word.
        cur = self.__con.cursor()
        
        matches = None
        rows = cur.execute("select word_id, doc_id, freq, position from word_posting \
                            where word_id='%s'" %word).fetchall()
        docs = [row[1] for row in rows]
        return docs
    
    def getDocsMulti(self, words):
        # return list of documents that contain every word in words.
        matches = set()
        for word in words:
            docs = set(self.getDocs(word))
            if not matches:
                matches = docs
            else:
                matches = matches.intersection(docs)
        return list(matches)
    
    def getWord(self, word):
        cur = self.__con.cursor()
        
        n_doc = 0
        total_freq = 0
        row = cur.execute("select word, n_doc, total_freq from word_list \
                           where word='%s'" %(word)).fetchone()
        if row:
            n_doc = row[1]
            total_freq = row[2]
            
        return n_doc, total_freq
    
    def getWordPosting(self, word, url):
        cur = self.__con.cursor()
        
        freq = 0
        position = []
        row = cur.execute("select word_id, doc_id, freq, position from word_posting \
                           where word_id='%s' and doc_id='%s'" 
                           %(word, url)).fetchone()
        if row:
            freq = row[2]
            position = [int(i) for i in row[3][1:-1].split(',')]
            
        return freq, position
    
    def updatePageRank(self, n_iter=20):
        print('update PageRank')
        self.__con.execute("delete from pagerank_list")
        
        cur = self.__con.cursor()
        docs = cur.execute("select rowid, doc from doc_list").fetchall()
        doc_list = [row[1] for row in docs]
        hash_doc_index = {doc:i for (i, doc) in enumerate(doc_list)}

        n = len(docs)
        A = np.zeros((n,n)) # Markov Matrix

        for doc in doc_list:
            doc_idx = hash_doc_index[doc]
            links = cur.execute("select from_doc_id, to_doc_id from link_list \
                                 where from_doc_id='%s'" %doc).fetchall()
            n_outlink = len(links)

            prob_stay = 1
            if n_outlink > 0:
                prob_out = 1/n_outlink
                for row2 in links:
                    to_doc = row2[1]
                    if to_doc in hash_doc_index:
                        to_doc_idx = hash_doc_index[to_doc]
                        A[doc_idx, to_doc_idx] += prob_out
                        prob_stay -= prob_out

            A[doc_idx, doc_idx] += prob_stay 
                # make A 'Markov matrix'. 
                # transfer probability of links to outside of doc_list 
                # to the original page.  

        B = np.ones((n,n)) / (n*np.ones((n,n)))
        A = A*0.85 + B*0.15
        
        Ak = A
        for i in range(n_iter-1):
            Ak = Ak.dot(A)

        pagerank_list = Ak[0] * 100
        print('Check Markov property: %.2f (should be 100.00)' %pagerank_list.sum())

        for doc, idx in hash_doc_index.items():
            rank = pagerank_list[idx]
            self.__con.execute("insert into pagerank_list (doc_id, pagerank) \
                                values ('%s', %.5f)" %(doc, rank))
            
    def getPageRank(self, url):
        cur = self.__con.cursor()
        
        pagerank = 0
        row = cur.execute("select doc_id, pagerank from pagerank_list \
                           where doc_id='%s'" %(url)).fetchone()
        if row:
            pagerank = row[1]
            
        return pagerank

In [3]:
class TextUtil:
    def getText(self, node):
        if node.name in ['head', 'script', 'style'] \
            or isinstance(node, Comment) \
            or isinstance(node, Doctype):
            return ''

        t = node.string
        if t:
            return re.sub('[\W]+', ' ', t).strip()
        else:
            ret = []
            for child in node.contents:
                t2 = self.getText(child)
                if t2 != '':
                    ret.append(t2)
            return ' '.join(ret)
   
    def preprocess(self, word):
        porter = PorterStemmer()
        stop = stopwords.words('english')

        word = word.lower().strip()
        if word in stop:
            return ''

        word = porter.stem(word)
        return word

    def tokenizer(self, words):
        ret = []
        for word in re.split('[\W]+', words):
            word = self.preprocess(word)
            if word != '':
                ret.append(word)
        return ret

In [4]:
class Logger:
    def __init__(self, file_name):
        self.f = open(file_name, 'w')
        
    def __del__(self):
        self.f.close()
        
    def log(self, s, stdout=False):
        self.f.write(s+'\n')
        if stdout:
            print(s)

In [5]:
class Crawler:
    def crawl(self, pages=[], depth=2, timeout=1):
        # define basic BFS of page graph using link. 
        # isValidUrl defines which pages to visit.
        
        log = Logger('search_engine_crawl.log')
        log.log('start crawl', stdout=1)
        
        t0 = time.time()
        t1 = time.time()
        
        dbcon = DBController()
        dbcon.initDB()

        for d in range(depth):
            log.log('\n\n depth: %d, pages: %d' %(d, len(pages)), stdout=1)
            
            cnt=0
            next_pages = []
            for url in pages:
                try:    
                    if dbcon.isErrorDoc(url) \
                        or dbcon.isDocIndexed(url) \
                        or not self.isValidUrl(url):
                        continue

                    # open page. skip non-html file. 
                    res = urlopen(url, timeout=timeout)
                    content_type = res.info()['Content-Type']
                    if not content_type.startswith('text/html'):
                        log.log('skip: %s. %s' %(content_type, url))
                        dbcon.addErrorDoc(url, content_type)
                        continue
                    
                    # parse html and add to index
                    soup = BeautifulSoup(res.read(), 'html.parser')
                    dbcon.addDoc(url)

                    # log
                    log.log('%5d sec (d=%.2f) | %5d. visiting %s' 
                             %(time.time()-t0, time.time()-t1, cnt, url))
                    cnt+=1
                    t1 = time.time()

                    # parge page text
                    text = TextUtil().getText(soup)
                    
                    # add word
                    words = TextUtil().tokenizer(text)
                    word_cnt = {} # {word: (freq, position)}
                    for idx, word in enumerate(words):
                        if word not in word_cnt:
                            word_cnt[word] = [0, []]
                        word_cnt[word][0] += 1
                        word_cnt[word][1].append(idx)

                    for word in word_cnt:
                        dbcon.addWord(word, url, word_cnt[word][0], word_cnt[word][1])

                    # parse links for BFS loop
                    link_url_list = []
                    for node in soup.find_all('a'):
                        link_text = TextUtil().getText(node) 
                        link_text = re.sub('[\s]+', ' ', link_text)
                        href = node.get('href')
                        if href and link_text:
                            link_url = urljoin(url, href)
                            link_url = link_url.split('#')[0]
                            if self.isValidUrl(link_url):
                                dbcon.addLink(url, link_url, link_text)
                                link_url_list.append(link_url)

                    next_pages += link_url_list
                except:
                    log.log("error: %s. url: %s" %(sys.exc_info()[1], url))
                    dbcon.addErrorDoc(url, sys.exc_info()[1])
            
            log.log(' visited pages: %d' %cnt, stdout=1)
            pages = next_pages
            
        log.log('finish crawl', stdout=1)
        
    def isValidUrl(self, url):
        if url.startswith('http'):
            return True
        return False
    
        
class BrownMathCrawler(Crawler):
    def isValidUrl(self, url):
        isBrownMath = False
        up = urlparse(url)
        if up and up.hostname and up.hostname.endswith('math.brown.edu'):
            isBrownMath = True
            
        if url.startswith('http') and isBrownMath:
            return True
        return False

In [6]:
class Ranker:
    def rank(self, docs, words):
        pass
    
class NoRanker(Ranker):
    def rank(self, docs, words):
        return [(doc, 0) for doc in docs]
    
class WordFreqRanker(Ranker):
    def rank(self, docs, words):
        doc_score = []
        dbcon = DBController()
        for doc in docs:
            total_freq = 0
            for word in words:
                freq, position = dbcon.getWordPosting(word, doc)
                total_freq += freq
            doc_score.append((doc, total_freq))
        doc_score = sorted(doc_score, key=lambda x: -x[1])
        return doc_score

class PageRankRanker(Ranker):
    def __init__(self):
        DBController().updatePageRank()

    def rank(self, docs, words):
        doc_score = []
        dbcon = DBController()
        for doc in docs:
            pagerank = dbcon.getPageRank(doc)
            doc_score.append((doc, pagerank))
        doc_score = sorted(doc_score, key=lambda x: -x[1])
        return doc_score

In [7]:
class SearchEngine:
    def __init__(self, ranker=PageRankRanker):
        self.ranker = ranker()
        
    def findMatches(self, query):
        dbcon = DBController()
        
        words = TextUtil().tokenizer(query)
        docs = dbcon.getDocsMulti(words)
        
        doc_score = self.ranker.rank(docs, words)
        return doc_score

# 1. Crawl Pages

In [8]:
crawler = BrownMathCrawler()

crawler.crawl(pages=['https://www.math.brown.edu', 
                     'https://www.math.brown.edu/faculty.php', 
                     'https://www.math.brown.edu/gradstuds.php'], 
              depth=5)

start crawl


 depth: 0, pages: 3
 visited pages: 3


 depth: 1, pages: 144
 visited pages: 89


 depth: 2, pages: 1749
 visited pages: 192


 depth: 3, pages: 2038
 visited pages: 387


 depth: 4, pages: 4715
 visited pages: 390
finish crawl


# 2. Explore Data

In [9]:
con = sqlite3.connect('search_engine.db')
c = con.cursor()

rows1 = c.execute("select * from doc_list").fetchall()
print(len(rows1))
rows2 = c.execute("select * from link_list").fetchall()
print(len(rows2))
rows3 = c.execute("select * from word_list").fetchall()
print(len(rows3))
rows4 = c.execute("select * from word_posting").fetchall()
print(len(rows4))

con.close()

1061
16631
11844
142789


In [10]:
df_doc = pd.DataFrame(rows1, columns=['doc'])
df_doc.sort_values(['doc'])[:5]

,doc
159,http://dug.math.brown.edu/
164,http://dug.math.brown.edu/contact
394,http://dug.math.brown.edu/home
163,http://dug.math.brown.edu/links-and-resources
162,http://dug.math.brown.edu/links-and-resources/...


In [12]:
df_link = pd.DataFrame(rows2, columns=['from_doc_id', 'to_doc_id', 'link_text'])
df_link.sort_values(['from_doc_id']).head()

,from_doc_id,to_doc_id,link_text
2471,http://dug.math.brown.edu/,http://dug.math.brown.edu/math-at-brown,Math at Brown
2485,http://dug.math.brown.edu/,http://dug.math.brown.edu/system/app/pages/rec...,Recent Site Activity
2484,http://dug.math.brown.edu/,http://dug.math.brown.edu/contact,go here
2483,http://dug.math.brown.edu/,http://dug.math.brown.edu/Introduction%20to%20...,here
2482,http://dug.math.brown.edu/,http://dug.math.brown.edu/Whim%20on%20Tourname...,here


In [13]:
df_word_list = pd.DataFrame(rows3, columns=['word', 'n_doc', 'total_freq'])
df_word_list.sort_values('n_doc', ascending=0)[:5]

,word,n_doc,total_freq
171,math,585,3683
97,brown,537,1689
112,mathemat,533,1944
804,1,482,3083
855,2,479,3085


In [14]:
df_word_posting = pd.DataFrame(rows4, columns=['word_id', 'doc_id', 'freq', 'position'])
df_word_posting.sort_values(['freq', 'word_id'], ascending=[0,0])[:5]

,word_id,doc_id,freq,position
33661,math,http://www.math.brown.edu/%7Ecalcplacement/Cal...,154,"[27, 34, 41, 46, 51, 93, 95, 100, 104, 119, 13..."
100211,math,https://www.math.brown.edu/~banchoff/CalcPlace...,154,"[27, 34, 41, 46, 51, 93, 95, 100, 104, 119, 13..."
121624,mathfrak,http://www.math.brown.edu/~jhs/MA0253/MA0253Ho...,144,"[6, 391, 397, 705, 709, 710, 713, 715, 718, 72..."
137606,1,http://www.math.brown.edu/~banchoff/midpoint/f...,136,"[5, 34, 47, 90, 136, 157, 161, 184, 187, 243, ..."
121498,k,http://www.math.brown.edu/~jhs/MA0253/MA0253Ho...,134,"[307, 316, 333, 352, 359, 372, 385, 417, 423, ..."


# 3. Search Pages

In [8]:
se = SearchEngine()

update PageRank
Check Markov property: 100.00 (should be 100.00)


In [9]:
se.findMatches('address of the department')[:5]

[('http://www.math.brown.edu/~jhs', 1.40686),
 ('http://www.math.brown.edu/~abrmovic/', 0.28716),
 ('https://www.math.brown.edu/contacts.html', 0.18715),
 ('http://www.math.brown.edu/~holmer/', 0.18683),
 ('http://www.math.brown.edu/~abrmovic', 0.17683)]

In [10]:
se.findMatches('admission graduate')[:5]

[('https://www.math.brown.edu/grad_prog.html', 0.21659),
 ('http://www.math.brown.edu/~wise/past_events.html', 0.09808),
 ('http://www.math.brown.edu/~mtassy', 0.08526),
 ('https://www.math.brown.edu/grad_prog_reqs.html', 0.07561),
 ('http://www.math.brown.edu/grad_prog.html', 0.05146)]

In [11]:
se.findMatches('calculus 2 requirements')[:5]

[('http://www.math.brown.edu/%7Ecalcplacement/CalcPlacement.html', 0.40501),
 ('https://www.math.brown.edu/course_desc.html', 0.39526),
 ('https://www.math.brown.edu/ugrad_prog.html', 0.22806),
 ('http://www.math.brown.edu/~abrmovic/MA/f1617/35', 0.19673),
 ('http://www.math.brown.edu/~mtchan/2016Spring_1040.html', 0.13292)]

In [12]:
se.findMatches('harmonic map')[:5]

[('https://www.math.brown.edu/course_desc.html', 0.39526),
 ('http://www.math.brown.edu/course_desc.html', 0.12977),
 ('http://www.math.brown.edu/~wongpin101/gradsem/seminarf13.html', 0.12418),
 ('https://www.math.brown.edu/phds.html', 0.12299),
 ('http://www.math.brown.edu/%7Etcanderson/Thesisfinal4.pdf', 0.11936)]

In [13]:
se.findMatches('search results should be empty for this query')[:5]

[]